In [36]:
import glob
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.metrics import silhouette_score, pair_confusion_matrix, confusion_matrix, ConfusionMatrixDisplay, accuracy_score, davies_bouldin_score, calinski_harabasz_score
from sklearn.model_selection import train_test_split, validation_curve, GridSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.neural_network import MLPClassifier
from sklearn.pipeline import Pipeline
from sklearn_som.som import SOM
from permetrics import ClusteringMetric

plt.style.use('default')
              

%run DataCleaningFunctions.ipynb

In [37]:
data = pd.read_csv("https://raw.githubusercontent.com/cdavidshaffer/CPSC4970-AI/master/data/penguins.csv")
data.info()
data.head(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 344 entries, 0 to 343
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   species            344 non-null    object
 1   island             344 non-null    object
 2   culmen_length_mm   344 non-null    object
 3   culmen_depth_mm    344 non-null    object
 4   flipper_length_mm  344 non-null    object
 5   body_mass_g        344 non-null    object
 6   sex                344 non-null    object
dtypes: object(7)
memory usage: 18.9+ KB


,species,island,culmen_length_mm,culmen_depth_mm,flipper_length_mm,body_mass_g,sex
0,Adelie,Torgersen,39.1,18.7,181.0,3750.0,MALE
1,Adelie,Torgersen,39.5,17.4,186.0,3800.0,FEMALE
2,Adelie,Torgersen,40.3,18.0,195.0,3250.0,FEMALE
3,Adelie,Torgersen,?,?,?,?,?
4,Adelie,Torgersen,36.7,19.3,193.0,3450.0,FEMALE
5,Adelie,Torgersen,39.3,20.6,190.0,3650.0,MALE
6,Adelie,Torgersen,38.9,17.8,181.0,3625.0,FEMALE
7,Adelie,Torgersen,39.2,19.6,195.0,4675.0,MALE
8,Adelie,Torgersen,34.1,18.1,193.0,3475.0,?
9,Adelie,Torgersen,42.0,20.2,190.0,4250.0,?


<h2><u>Data Cleaning</u></h2>

In [38]:
# check text column unique values
textual_columns = ['species', 'island', 'sex']
for column in textual_columns:
        column_values = data[column].unique()
        print(f"{column.title()} Values: ", *column_values)

Species Values:  Adelie Chinstrap Gentoo
Island Values:  Torgersen Biscoe Dream
Sex Values:  MALE FEMALE ? _


In [14]:
# Replace invalid text entries
replace_characters = ['?', '_']

for char in replace_characters:
    data['sex'].replace(char, None, inplace=True)
    
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 344 entries, 0 to 343
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   species            344 non-null    object
 1   island             344 non-null    object
 2   culmen_length_mm   344 non-null    object
 3   culmen_depth_mm    344 non-null    object
 4   flipper_length_mm  344 non-null    object
 5   body_mass_g        344 non-null    object
 6   sex                333 non-null    object
dtypes: object(7)
memory usage: 18.9+ KB


/var/folders/0_/nmh5w64x3q5_ffwfkr35sscw0000gn/T/ipykernel_11147/4163542533.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data['sex'].replace(char, None, inplace=True)


In [15]:
# convert text columns to string type
for i in textual_columns:
    data[i] = data[i].astype('string')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 344 entries, 0 to 343
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   species            344 non-null    string
 1   island             344 non-null    string
 2   culmen_length_mm   344 non-null    object
 3   culmen_depth_mm    344 non-null    object
 4   flipper_length_mm  344 non-null    object
 5   body_mass_g        344 non-null    object
 6   sex                333 non-null    string
dtypes: object(4), string(3)
memory usage: 18.9+ KB


In [ ]:
# clean numerical columns from non-numerical values
numerical_columns = ['culmen_length_mm', 'culmen_depth_mm', 'flipper_length_mm', 'body_mass_g']
for column in numerical_columns:
        data[column] = pd.to_numeric(data[column], errors='coerce')

data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 344 entries, 0 to 343
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   species            344 non-null    string 
 1   island             344 non-null    string 
 2   culmen_length_mm   342 non-null    float64
 3   culmen_depth_mm    342 non-null    float64
 4   flipper_length_mm  342 non-null    float64
 5   body_mass_g        342 non-null    float64
 6   sex                333 non-null    string 
dtypes: float64(4), string(3)
memory usage: 18.9 KB


In [17]:
# Drop all rows with missing values
data = data.dropna()
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 333 entries, 0 to 343
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   species            333 non-null    string 
 1   island             333 non-null    string 
 2   culmen_length_mm   333 non-null    float64
 3   culmen_depth_mm    333 non-null    float64
 4   flipper_length_mm  333 non-null    float64
 5   body_mass_g        333 non-null    float64
 6   sex                333 non-null    string 
dtypes: float64(4), string(3)
memory usage: 20.8 KB


In [18]:
X = data.drop("species", axis=1)
y = data["species"]

In [21]:
total = len(y)
print(f"Total: {total}")
penguins = y.unique()
for penguin in penguins:
    species = y[y == penguin]
    count = species.value_counts().sum()
    species_fraction = count / total
    print(f"Number of {penguin}'s: ", count)
    print(f"Fraction of {penguin}'s: ", "{:.2f}%".format(species_fraction * 100))

Total: 333
Number of Adelie's:  146
Fraction of Adelie's:  43.84%
Number of Chinstrap's:  68
Fraction of Chinstrap's:  20.42%
Number of Gentoo's:  119
Fraction of Gentoo's:  35.74%


<h3><u>Data Preparation</u></h3>

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
display(X_train.shape) 
display(X_test.shape)
display(y_train.shape)
display(y_test.shape)

(266, 6)

(67, 6)

(266,)

(67,)

In [10]:
vc_dict = {}
lc_dict = {}

<h2><strong><u>Data Preprocessing</u></strong></h2>

<h3><u>Feature Engineering (SOM)</u></h3>

In [ ]:
# separate numerical and non-numerical data
X_train_nonnumerical = X_train.drop(columns=numerical_columns)
X_test_nonnumerical = X_test.drop(columns=numerical_columns)
display(X_train_nonnumerical.describe())
display(X_test_nonnumerical.describe())

X_train_numerical = X_train[numerical_columns]
X_test_numerical = X_test[numerical_columns]
display(X_train_numerical.describe())
display(X_test_numerical.describe())

,island,sex
count,266,266
unique,3,2
top,Biscoe,MALE
freq,128,142


,island,sex
count,67,67
unique,3,2
top,Biscoe,FEMALE
freq,35,41


,culmen_length_mm,culmen_depth_mm,flipper_length_mm,body_mass_g
count,266.000000,266.000000,266.000000,266.000000
mean,44.497744,17.134962,201.808271,4253.289474
std,5.365930,2.020036,13.755167,793.770550
min,34.000000,13.100000,172.000000,2700.000000
25%,39.725000,15.300000,190.250000,3650.000000
50%,45.350000,17.450000,198.000000,4100.000000
75%,49.075000,18.700000,214.000000,4850.000000
max,59.600000,21.500000,230.000000,6300.000000


,culmen_length_mm,culmen_depth_mm,flipper_length_mm,body_mass_g
count,67.000000,67.000000,67.000000,67.000000
mean,41.988060,17.283582,197.626866,4023.507463
std,5.451731,1.762109,14.638923,829.976732
min,32.100000,13.700000,174.000000,2850.000000
25%,37.700000,16.250000,187.500000,3375.000000
50%,41.300000,17.200000,193.000000,3800.000000
75%,45.750000,18.500000,207.500000,4612.500000
max,54.300000,21.100000,231.000000,5950.000000


In [ ]:
som = SOM(m=5, n=5, dim=X_train_numerical.shape[1], random_state=42)
som.fit(X_train_numerical.values)
som_train_predictions = som.predict(X_train_numerical.values)
som_test_predictions = som.predict(X_test_numerical.values)

In [ ]:
X_train_nonnumerical_som = pd.DataFrame(np.append(X_train_nonnumerical, som_train_predictions.reshape(-1, 1), axis=1), columns=X_train_nonnumerical.columns.tolist() + ['SOM Category'])
X_test_nonnumerical_som = pd.DataFrame(np.append(X_test_nonnumerical, som_test_predictions.reshape(-1, 1), axis=1), columns=X_train_nonnumerical.columns.tolist() + ['SOM Category'])


display(X_train_nonnumerical_som.shape)
display(X_test_nonnumerical_som.shape)
display(X_train_nonnumerical_som.columns.tolist())
display(X_test_nonnumerical_som.columns.tolist())
display(X_train_nonnumerical_som.head(5))
display(X_test_nonnumerical_som.head(5))
preprocessor = make_column_transformer(
    (OneHotEncoder(), X_train_nonnumerical_som.columns.tolist()),
    remainder='passthrough'
)
X_train_processed = preprocessor.fit_transform(X_train_nonnumerical_som)
X_test_processed = preprocessor.transform(X_test_nonnumerical_som)
display(X_train_processed.shape)
display(X_test_processed.shape)


(266, 3)

(67, 3)

['island', 'sex', 'SOM Category']

['island', 'sex', 'SOM Category']

,island,sex,SOM Category
0,Biscoe,MALE,8
1,Dream,FEMALE,23
2,Biscoe,MALE,8
3,Biscoe,MALE,4
4,Dream,FEMALE,0


,island,sex,SOM Category
0,Biscoe,FEMALE,10
1,Biscoe,FEMALE,21
2,Biscoe,MALE,4
3,Biscoe,MALE,11
4,Biscoe,MALE,11


(266, 30)

(67, 30)